In [1]:
import os
raw = os.getcwd()
import sys
import re
import import_ipynb
import numpy as np
import pandas as pd
import time
import jieba
from jieba import lcut
sys.path.append("C://Users//admin//Desktop//project//ChatBot_threebotbyJimmy//")
import FAQBot_v2 as FAQBot

#sklearn
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression

#bert
from sentence_transformers import SentenceTransformer

importing Jupyter notebook from FAQBot_v2.ipynb


In [3]:
os.chdir('C://Users//admin//Desktop//aicode-master//')
#textlearn
#sys.path.append('C://Users//admin//Desktop//aicode-master//')
from textLearn.bin.initializer import initializer
from textLearn.bin.classification import textClassification
from user_config import user
from textLearn.bin.IO import modelLoader


wd = 'C://Users//admin//Desktop//aicode-master//'
os.chdir(raw)

# Task Bot

In [ ]:
#到帐问题 0
d0 = {'提币没到': '您好，小U稍后为您转接客服小姐姐为您处理。请您提供【订单号码】给客服小姐姐哦~ #转人工#',
      '附言': "亲，只要付款金额和订单金额一致，系统会自动到账哦!若您还是需要客服小姐姐说明，请点击或输入[link submit='到账问题转人工']转人工[/link]~小U为您转接客服姐姐解决您的问题呦~",
      '到账问题转人工': '亲，小U马上为您转接客服小姐姐为您处理~#转人工#',
      '充币没到': '您好，小U稍后为您转接客服小姐姐为您处理。请您提供【订单号码】、【转账成功的截图】、【Upay ID】给客服小姐姐哦~ #转人工#',
      '提款卖币没到帐': '您好，小U稍后为您转接客服小姐姐为您处理。请您提供【订单号码】给客服小姐姐哦~ #转人工#',
      '存款买币没到帐': '小主您好，卖家收到款项，确认收款后是自动到账的，如果长时间未到账，小U稍后为您转接客服小姐姐为您处理。请您提供提供【订单号】，【upay订单截图】，以及【支付转账成功的截图】给客服小姐姐哦~ #转人工#'}

#提款卖币问题 1
d1 = {'手续费': '小主您好~接收其他平台的USDT不需要手续费，内部转账也无需手续费，若进行外部转账则会产生手续费的哦，公链会根据繁忙程度收取五个币起的手续费哦~',
     '提款支持方式': '亲，目前出售暂仅支持银行卡哦~',
     '汇率': '您好，购买和出售是按照市场汇率波动实时变动的哦，upay钱包无法操控。汇率波动造成您的资产金额有偏差，您不必担心，您的资产是安全的，还麻烦您随时关注汇率哦~',
     '资金冻结': '亲，冻结资金是未完成的订单金额，请不用担心哦。',
     '余额不足': '亲，您可进行购买币，再一并出售或提币。',
     '最低卖币额度': '小主您好，最低出售金额为16个USDT哦~',
     '提币等候时间': "您好，提币后订单会在一个小时内到账，麻烦您耐心等待哦，如您长时间未到账，麻烦您提供您的【提币订单号码】给客服小姐姐帮您查询哦~点击[link submit='提款卖币转人工']●转人工[/link]为您转接客服小姐姐~",
     '忘记提币密码': '小主您好，您可以点击 【我的】-【安全中心】-【法币资金密码】，可以重新设置哦~小提示：修改提币密码是不需要原密码的哦!',
     '卖币等候时间': "亲，一般成功操作出售后，系统会在2小时内自动处理到账的呢，如2小时后仍未到账，您可再次访问我们24小时在线客服哦，谢谢~！点击[link submit='提款卖币转人工']●转人工[/link]为您转接客服小姐姐~",
     '最高卖币额': '小主您好，单笔卖币最大额度约为 7000 USDT(依照实时汇率会有变动)\xa0，出售到银行卡收款最高不可超过四万五千元人民币，每日交易无限额，请您放心使用哦~*提醒您，最多可下四笔订单，等待订单完成后还可以继续下单，感谢您的支持',
     '最低提币额度': '小主您好，目前提币最低限额为 10 USDT哦~',
     '如何提币到游戏平台': '打开Upay钱包选择【我的】-【提币】-【USDT币种】，在游戏官网复制的USDT地址粘贴到提币地址，输入存款金额即可点击提币后输入【法币资金密码】即可成功存款了。',
     '如何卖': '小主您好，目前仅可提款至银行卡，您可以根据以下步骤进行操作卖币：\n在【委托】页面点击出售\n输入需要出售的金额\n选择银行卡信息，点击【出售】即可\xa0'}


#存款买币问题 2
d2 = {'支持什么类型的USDT': '小主，目前仅支持ERC20类型的USDT交易，其他类型的USDT无法到账哦!',
      '最多可购买金额': '小主您好，单笔购买目前最高是6970 USDT哦~谢谢您的支持',
      '怎么购买USDT': '您可以根据以下步骤进行操作买币：\n1. 在【委托】页面点击购买\n2. 输入需要购买的金额，点击购买\n3. 选择银行卡信息，点击确认购买，点击去付款\n4.按照订单上的收款账户和金额可在网银 支付宝 微信 云闪付进行转账，转账后点击订单上面的完成付款哦',
      '怎么充币': '如您需要从别的地方转钱进来Upay钱包，您可点击【我的】--【充币】页面进行查看您的钱包地址，是用于收款的。',
      '无法购买USDT': '选择是何种付款方式',
      '付款方式有哪些': '购买付款方式支持微信、支付宝、云闪付和网银进行转账的呢。微信和支付宝比较严格，一般建议使用网银。',
      '买币金额输错了': '亲，麻烦请您提供一下【UPAY订单截图】【UPAY订单号】和【成功付款截图】以上信息提供齐全，这边才能帮您查询哦~#转人工#\n温馨提醒您：在操作买币时，付款金额与订单金额必须一致呢，以免因此导致到账延迟 给您带来不便，谢谢！',
      '最低可购买金额': '亲，Upay钱包最低购买是100人民币哦。'}

#银行卡问题 3
d3 = {'绑银行卡': '小主您好，您可以根据以下步骤进行绑定银行卡信息：\n1. 在【我的】页面，点击【安全中心】\n2.在【安全中心】页面点击【收款方式管理】\n3.将出现【收款账户】页面，点击添加\n4.您可以选择需要绑定的选项进行绑定\n温馨提醒：【目前支付宝微信维护中，暂时不支持哦!】',
     '解绑银行卡': '小主，银行卡无法解绑，但您可以添加新的银行卡哦~',
     '支持哪些银行卡': '您好，卖币目前仅支持绑定以下银行：农业银行，中国银行，建设银行，招商银行，交通银行，民生银行，平安银行，中信银行，光大银行，中国邮政储蓄银行，兴业银行，华夏银行，广东发展银行，上海浦东发展银行，东亚银行，北京银行，北京农村商业银行，上海银行，浙商银行，渤海银行，杭州银行，南京银行，上海农村商业银行，青岛银行',
     '可以使用他人的银行卡吗': '小主您好，可以使用其他人的银行卡哦!您只需要按照订单对应的金额，收款人，卡号付款即可~很方便的!',
     '银行卡异常': "亲，微信和支付宝比较严格，建议使用网银进行操作呢!或者重新下单匹配新的收款账户哦~若您还是需要客服小姐姐说明，请点击或输入[link submit='银行卡转人工']转人工[/link]~小U为您转接客服姐姐解决您的问题呦~",
     '如何提款到银行卡': '亲，您点击【委托】，点【出售】就可以添加到银行卡哦。'}

#操作问题 4
d4 ={'实名认证在哪操作': '亲，请您登录Upay钱包，点击【我的】--【账户中心】--【收款方式管理】页面进行操作即可哦。',
 '如何绑定手机号': '小主您好，您可以点击-【我的】-【个人中心】-【手机号】进行验证后绑定哦~',
 '忘记Upay登录帐号': '小主，UPAY账号是通过手机号码或邮箱注册的，您可以尝试通过手机号码和注册邮箱找回密码后查看账号。\n若您无法想起，建议您重新注册账号。给您带来不便，敬请谅解。',
 '如何设定法币资金密码': '小主您好，您可以点击 【我的】-【安全中心】-【法币资金密码】，即可设置法币资金密码哦~',
 '忘记Upay登录密码': '小主您好，忘记登录密码不用着急~请您在登录页面点击【忘记密码】，按照以下步骤就可以了哦~\n1、手机号找回：输入 注册手机号+新密码+手机收到的验证码，即可修改新密码\u200b\n2、注册邮箱找回：输入 注册邮箱账号+新密码+邮箱收到验证码\u200b，即可修改新密码\u200b',
 '忘记法币资金密码': "请登录Upay钱包，点击 我的--安全中心--法币资金密码页面进行操作即可。小提示：修改提币密码是不需要原密码的哦!\n或点击[link submit='提款卖币转人工']转人工[/link]请客服小姐姐为您说明哦!",
 'Upay邀请码问题': '小主，Upay没有邀请码，您可以在我的分享应用中找到连接或二维码复制给您好友哦!',
 'Upay ID在哪里': '小主，Upay ID在钱包主界面中“我的”左上角哦~',
 'Upay ID是什么': '小主，Upay ID是您系统自动给您生成身份标识号码呢!',
 '手机号可以更改吗': '小主您好，抱歉目前Upay暂时不支持更改手机号功能哦~',
 '账户可以注销删除吗': '小主您好，抱歉，目前upay暂时不支持注销账户功能哦，感谢您的支持~',
 '怎么注册': '小主您好，请您按照以下步骤进行注册哦~\n1、个人使用的手机号码+密码+手机验证码 直接验证登录\u200b\n2、个人使用的邮箱+密码 通过邮箱收取链接点击链激活',
 'Upay验证码问题': '小U帮您转接客服小姐姐，您再跟他说您是注册还是设置提币密码收不到验证码哦~#转人工#',
 '钱包地址在哪里查看': '亲，请登录Upay钱包，点击【我的】--【充币】页面进行查看您的钱包即可呢。',
 'network error': '请您卸载旧版本，下载新的app哦~app下载请点击APP下载',
 '维护公告': '亲，小U马上为您转接客服小姐姐为您说明维护状况~#转人工#',
 '请检查网路': '小主，建议请下载一下我们最新版的APP呢!app下载链接：https://h5.upaycloud.com/app.html?go',
 '闪退': '小主，这是您程序掉签，建议请重新下载一下我们最新版的APP呢!app下载链接：https://h5.upaycloud.com/app.html?go',
 '官网网址': '小主您好，UPAY 官网请点击：https://www.upay-coin.com/',
 '查看交易记录': '小主您好，登陆成功后，点击页面下方交易记录按钮即可查询。您也可以通过右上角筛选功能快速查看交易信息哦!'}


#订单取消问题 5
d5 = {'我想要取消购买订单': '小主，小U请漂亮的客服姐姐来帮您，麻烦您提供您【订单编号】，以便客服姐姐为您查看呢~#转人工#',
 '购买订单被取消了': '小主，小U请漂亮的客服姐姐来帮您，麻烦您提供您【订单编号】【游戏账号】【存款截图】，以便客服姐姐为您查看呢~#转人工#',
 '我想要取消提币': '小主，小U请漂亮的客服姐姐来帮您查看呢~#转人工#',
 '我想要取消出售订单': '小主，小U请漂亮的客服姐姐来帮您，麻烦您提供您【订单编号】，以便客服姐姐为您查看呢~#转人工#',
 '出售订单被取消了': '小主，小U请漂亮的客服姐姐来帮您，麻烦您提供您【订单编号】，以便客服姐姐为您查看呢~#转人工#'}

In [ ]:
'''
#使用者场景，可能会遇到(待验证)：
#a.同意词可能需要不同回答
#b.不同意但需要相同回答-->没问题
#c.节点太多，可能会分错-->待验证
#d.客户想要的跟你的分法不同  例如你的演算判断是A  客户说是B

#存款买币问题 2
d2 = {'支持什么类型的USDT': '小主，目前仅支持ERC20类型的USDT交易，其他类型的USDT无法到账哦!',
      '最多可购买金额': '小主您好，单笔购买目前最高是6970 USDT哦~谢谢您的支持',
      '怎么购买USDT': '您可以根据以下步骤进行操作买币：\n1. 在【委托】页面点击购买\n2. 输入需要购买的金额，点击购买\n3. 选择银行卡信息，点击确认购买，点击去付款\n4.按照订单上的收款账户和金额可在网银 支付宝 微信 云闪付进行转账，转账后点击订单上面的完成付款哦',
      '怎么充币': '如您需要从别的地方转钱进来Upay钱包，您可点击【我的】--【充币】页面进行查看您的钱包地址，是用于收款的。',
      '无法购买USDT': '选择是何种付款方式',
      '付款方式有哪些': '购买付款方式支持微信、支付宝、云闪付和网银进行转账的呢。微信和支付宝比较严格，一般建议使用网银。',
      '买币金额输错了': '亲，麻烦请您提供一下【UPAY订单截图】【UPAY订单号】和【成功付款截图】以上信息提供齐全，这边才能帮您查询哦~#转人工#\n温馨提醒您：在操作买币时，付款金额与订单金额必须一致呢，以免因此导致到账延迟 给您带来不便，谢谢！',
      '最低购买金额': '亲，Upay钱包最低购买是100人民币哦。'}

customer_question = '最少购买金额'
test_embeddings = FAQBot.FAQ_model.encode(customer_question)
similarity = cosine_similarity([test_embeddings], 
                               sentence_embeddings)

q1 = list(d2.keys())
a1 = list(d2.values())
max_cos_similar = np.max(similarity)
if max_cos_similar >= 0.8:
    max_idx = np.argmax(similarity)
    most_likely_question = q1[max_idx]    
    most_likely_ans = a1[max_idx]
    res = 1
else:
    most_likely_question = '这没有设置节点'
    most_likely_ans = '这没有设置节点_所以没答案'
    res = 0
probsarg = [x for x in similarity.argsort()[0][-3:][::-1] ] 
possibe_question = [q1[j] for j in probsarg]
print('客户问的问题:  '+customer_question, 
      '最可能的问题：  '+most_likely_question, 
      '最可能问题的答案:  '+most_likely_ans, 
      '可能的问题:  ' +str(possibe_question), sep='\n')
'''

In [ ]:
'''
df0 = pd.DataFrame({'节点':['到帐问题']*len(d0), '标准问题':list(d0.keys()), '回答':list(d0.values())})
df1 = pd.DataFrame({'节点':['提款卖币问题']*len(d1), '标准问题':list(d1.keys()), '回答':list(d1.values())})
df2 = pd.DataFrame({'节点':['存款买币问题']*len(d2), '标准问题':list(d2.keys()), '回答':list(d2.values())})
df3 = pd.DataFrame({'节点':['银行卡问题']*len(d3), '标准问题':list(d3.keys()), '回答':list(d3.values())})
df4 = pd.DataFrame({'节点':['操作问题']*len(d4), '标准问题':list(d4.keys()), '回答':list(d4.values())})
df5 = pd.DataFrame({'节点':['订单取消问题']*len(d5), '标准问题':list(d5.keys()), '回答':list(d5.values())})

task_qa_ = pd.concat([df0, df1, df2, df3, df4, df5], axis=0)

writer = pd.ExcelWriter('C://Users//user//Desktop//task_qa_.xlsx', engine='openpyxl', index=0)

# df 指要寫出的內容
task_qa_.to_excel(writer, sheet_name='Sheet1', index=False)
writer.save()
'''
task_qa_dict = {}
task_qa_dict[0] = d0
task_qa_dict[1] = d1
task_qa_dict[2] = d2
task_qa_dict[3] = d3
task_qa_dict[4] = d4
task_qa_dict[5] = d5

In [ ]:
init = initializer("5712")
tc = textClassification(init)

#model up
LRModel = {}
for u in user:
    LRModel[u] = modelLoader(wd + "/textLearn/models/LR/" + u + "/LR_model")
#label dict
change_dict = {'到帐问题':0, '提款卖币问题':1, '存款买币问题':2, '银行卡问题':3, '操作问题':4, '订单取消问题':5, '其他意图':6}
change_dict_verse = {v: k for k, v in change_dict.items()}

In [ ]:
def TASK_Bot_first(text):
    Task_first = tc.predict(text, LRModel['UP'], product="UP", user="UP")
    label = int(Task_first['label'])-1
    sentiment = Task_first['sentiment']['compalin_detail']
    if sentiment['complain'] == False:
        emotion = '无情绪'
    else:
        if sentiment['normal_complain']:
            emotion = '一般抱怨'
        elif sentiment['serious_complain']:
            emotion = '严重抱怨'
        elif sentiment['losing_money']:
            emotion = '输钱'
        elif sentiment['urging']:
            emotion = '催促'    
    return label, emotion

In [ ]:
def TASK_Bot_two(text, label, threshold, control):
    if label == 6:
        #chat bot
        most_likely_ans, res, most_likely_question, possibe_question, top4_similarty =  'chatbot',0, 'chatbot', 'chatbot', 'chatbot'
    else:
        dict_down = task_qa_dict[label]
        question = list(dict_down.keys())
        ans = list(dict_down.values())
        sentence_emb = FAQBot.FAQ_model.encode(question)
        test_emb = FAQBot.FAQ_model.encode(text)
        similarity = cosine_similarity([test_emb], sentence_emb)       
        probsarg = [x for x in similarity.argsort()[0][-4:][::-1] ] 
        possibe_question = [question[j] for j in probsarg][1:]
        top4_similarty = [similarity[0][j] for j in probsarg]
        
        
        max_cos_similar = np.max(similarity)
        if max_cos_similar >= threshold:
            max_idx = np.argmax(similarity)
            most_likely_question = question[max_idx]    
            most_likely_ans = ans[max_idx]
            res = 1
        else:
            most_likely_question = '这没有设置节点'
            most_likely_ans = '这没有设置节点_所以没答案'
            res = 2 
    if control == 'jy_test':
        return most_likely_ans, res, most_likely_question, possibe_question, top4_similarty    
    elif control == 'prod':
        return most_likely_ans, res

In [ ]:
'''
s = time.time()
text = '我想要提款'
label, emotion = TASK_Bot_first(text)
TASK_Bot_two(text, label, 0.7, control='jy_test')
#print(time.time()-s)
'''

In [ ]:
'''
#使用者场景，可能会遇到(待验证)：
#a.同意词可能需要不同回答
#b.不同意但需要相同回答-->没问题
#c.节点太多，可能会分错-->待验证
#d.客户想要的跟你的分法不同  例如你的演算判断是A  客户说是B

#存款买币问题 2
d2 = {'支持什么类型的USDT': '小主，目前仅支持ERC20类型的USDT交易，其他类型的USDT无法到账哦!',
      '最多可购买金额': '小主您好，单笔购买目前最高是6970 USDT哦~谢谢您的支持',
      '怎么购买USDT': '您可以根据以下步骤进行操作买币：\n1. 在【委托】页面点击购买\n2. 输入需要购买的金额，点击购买\n3. 选择银行卡信息，点击确认购买，点击去付款\n4.按照订单上的收款账户和金额可在网银 支付宝 微信 云闪付进行转账，转账后点击订单上面的完成付款哦',
      '怎么充币': '如您需要从别的地方转钱进来Upay钱包，您可点击【我的】--【充币】页面进行查看您的钱包地址，是用于收款的。',
      '无法购买USDT': '选择是何种付款方式',
      '付款方式有哪些': '购买付款方式支持微信、支付宝、云闪付和网银进行转账的呢。微信和支付宝比较严格，一般建议使用网银。',
      '买币金额输错了': '亲，麻烦请您提供一下【UPAY订单截图】【UPAY订单号】和【成功付款截图】以上信息提供齐全，这边才能帮您查询哦~#转人工#\n温馨提醒您：在操作买币时，付款金额与订单金额必须一致呢，以免因此导致到账延迟 给您带来不便，谢谢！',
      '最低购买金额': '亲，Upay钱包最低购买是100人民币哦。'}

customer_question = '金额输少了'
test_embeddings = FAQBot.FAQ_model.encode(customer_question)
sentence_embeddings = FAQBot.FAQ_model.encode(list(d2.keys()))
similarity = cosine_similarity([test_embeddings], 
                               sentence_embeddings)

q1 = list(d2.keys())
a1 = list(d2.values())
max_cos_similar = np.max(similarity)
if max_cos_similar >= 0.8:
    max_idx = np.argmax(similarity)
    most_likely_question = q1[max_idx]    
    most_likely_ans = a1[max_idx]
    res = 1
else:
    most_likely_question = '这没有设置节点'
    most_likely_ans = '这没有设置节点_所以没答案'
    res = 0
probsarg = [x for x in similarity.argsort()[0][-3:][::-1] ] 
possibe_question = [q1[j] for j in probsarg]
print('客户问的问题:  '+customer_question, 
      '最可能的问题：  '+most_likely_question, 
      '最可能问题的答案:  '+most_likely_ans, 
      '可能的问题:  ' +str(possibe_question), sep='\n')
'''